In [1]:
import kfbReader as kr
import torch
import torchvision
import cv2
import numpy as np
import pandas as pd
import os
import time
import random
import copy
from Data_PreProcess import Data_preprocess as dp
import json
from data_loader import DataGenerate,collate
from torch.utils.data import Dataset, DataLoader
from Generate_Anchor import Generate_Anchor
from torchvision import models
from Eval import eval_model
from Net3 import Net

In [2]:
def collate_sub(batch):
    img = torch.from_numpy(batch[0][0].transpose(2, 0, 1)[np.newaxis,]).cuda().float()
    return img, batch[0][1], batch[0][2]


class SubGenerate(Dataset):
    def __init__(self, imgpath, input_size):
        self.imgpath = imgpath
        self.input_size = input_size
        self.reader = kr.reader()
        self.reader.ReadInfo(imgpath, 20, True)
        self.Width = self.reader.getWidth()
        self.Height = self.reader.getHeight()
        self.stride = int(input_size * 0.85)
        self.index_w -= self.stride
        self.index_h = 0

    def __len__(self):
        length = ((self.Width - self.input_size) // self.stride + 1) * ((self.Height - self.input_size) // self.stride + 1)
        self.Width -= self.input_size
        self.Height -= self.input_size
        # print("len = ", length)
        return length

    def __getitem__(self, index):
        self.index_w += self.stride
        if (self.index_w + self.input_size > self.Width):
            self.index_w = 0
            self.index_h += self.stride
        img = self.reader.ReadRoi(self.index_w, self.index_h, self.input_size, self.input_size, 20)
        return img, self.index_w, self.index_h


def collate(batch):
    return batch[0][0], batch[0][1]


class InferGenerate(Dataset):
    def __init__(self, data_path, input_size):
        self.data_path = data_path
        self.datalist = os.listdir(data_path)
        self.input_size = input_size

    def __len__(self):
        return len(self.datalist)

    def __getitem__(self, index):
        data_name = self.datalist[index]
        data_path = os.path.join(self.data_path, data_name)
        sub_generater = DataLoader(SubGenerate(imgpath=data_path, input_size=self.input_size), \
                                   batch_size=1, drop_last=False, collate_fn=collate_sub)
        return sub_generater, data_name


data_path = r"E:\cancer\test_1"
input_size = 864

# 2阶段 预测
model = Net(input_size=input_size, batch_size=1, is_training=False)
model.load_state_dict(torch.load(r"E:\cancer\model\model_include3\eps=3.t7"), strict=False)
#model = model.cuda()
model.eval()
model = model.cuda()
infer_generate = DataLoader(InferGenerate(data_path=data_path, input_size=input_size), \
                            batch_size=1, drop_last=False, collate_fn=collate)
with torch.no_grad():
    for idx, dat in enumerate(infer_generate):
        dat_name = dat[1]
        js = []
        start = time.time()
        for img, w, h in dat[0]:
            _, s2_res = model(img)
            del img
            if(s2_res.size(0) == 0):continue
            s2_res[:, 2:4] -= s2_res[:, :2]
            s2_res[:, 0] += w
            s2_res[:, 1] += h
            bboxs = s2_res.detach().cpu().numpy().astype(np.float)
            for th in range(bboxs.shape[0]):
                bbox = bboxs[th]
                js.append({"x": bbox[0], "y": bbox[1], "w": bbox[2], "h": bbox[3], "p": bbox[4]})
            #print("total time : %.5f", time.time() - start)
        with open(r"D:\ans\%s.json" % dat_name[:-4], 'w') as f:
            json.dump(js, f)
        print("No%d process, time = %f" % (idx, time.time() - start))

No0 process, time = 555.779760
No1 process, time = 577.118582
No2 process, time = 537.133116
No3 process, time = 811.378129
No4 process, time = 750.211802
No5 process, time = 571.871576
No6 process, time = 633.141597
No7 process, time = 450.841552
No8 process, time = 559.430474
No9 process, time = 567.413714
No10 process, time = 606.343294
No11 process, time = 550.254420
No12 process, time = 445.848145
No13 process, time = 651.923406
No14 process, time = 713.654817
No15 process, time = 593.925253
No16 process, time = 728.288754
No17 process, time = 529.276242
No18 process, time = 626.319445
No19 process, time = 239.542349
No20 process, time = 681.694781
No21 process, time = 223.745318
No22 process, time = 666.140396
No23 process, time = 596.116082
No24 process, time = 868.616122
No25 process, time = 564.120954
No26 process, time = 597.020490
No27 process, time = 361.696363
No28 process, time = 777.556842
No29 process, time = 562.692153
No30 process, time = 647.217047
No31 process, time

In [ ]:
# 1阶段 预测
# 用于 retinanet
data_path = r"E:\cancer\neg_1"
datalist = os.listdir(data_path)

input_size = 2560
prior_anchor = get_prior_anchor(input_size = input_size)
with torch.no_grad():
    for idx,dat_name in enumerate(datalist):
        start = time.time()
        reader = kr.reader()
        reader.ReadInfo(os.path.join(data_path, dat_name),20,True)
        width = reader.getWidth() - input_size
        height = reader.getHeight() - input_size
        js = []
        for h in range(0,height,input_size):  #后续改成 d = self.input_size/2 
            for w in range(0,width,input_size):
                #cv2.waitKey(100)
                img = reader.ReadRoi(w,h,input_size,input_size,20)
                img = img.transpose(2,0,1)[np.newaxis,]
                img = torch.from_numpy(img).cuda().float()
                pred_cls,pred_loc = model(img)
                index = pred_cls>0.9
                if (torch.sum(index) > 0) and (torch.sum(index) < 2000):
                    scores = pred_cls[index]
                    pred_loc = pred_loc[index]
                    src_anchor = prior_anchor[index]
                    x = pred_loc[:,0] * src_anchor[:,2] + src_anchor[:,0] + w
                    y = pred_loc[:,1] * src_anchor[:,3] + src_anchor[:,1] + h
                    _w = torch.exp(pred_loc[:,2]) * src_anchor[:,2]
                    _h = torch.exp(pred_loc[:,3]) * src_anchor[:,3]
                    pred = torch.stack((x,y,_w,_h,scores),axis=1).detach().cpu().numpy()
                    pred = np.clip(pred,0,None)
                    pred[:,2:4] += pred[:,:2]
                    index = dp.py_cpu_nms(pred)
                    bboxs = pred[index].astype(np.float)
                    bboxs[:,2:4] -= bboxs[:,:2]
                    for th in range(bboxs.shape[0]):
                        bbox = bboxs[th]
                        js.append({"x":bbox[0], "y":bbox[1], "w":bbox[2], "h":bbox[3], "p":bbox[4]})
        with open(r"E:\cancer\neg_label\neg_label_1\%s.json"%dat_name[:-4],'w') as f:
            json.dump(js,f)
        print("No%d process, time = %.3f"%(idx, time.time() - start))